In [1]:
import pandas as pd
import re
from urllib.parse import urlparse 

In [2]:
df = pd.read_csv('/Users/andrewkerr/Downloads/DC_lead_pipe_dataset.csv')
relevant_df = df[['PremiseAddress', 'PublicSymbol', 'PublicServiceMaterial', 'PublicServiceMaterialDescription', 'PrivateSymbol', 'PrivateServiceMaterial', 'PrivateServiceMaterialDescription']]
df = relevant_df
df = df[(df['PublicSymbol'] != 'Unknown') & (df['PrivateSymbol'] != 'Unknown')]
df = df[['PremiseAddress', 'PublicSymbol', 'PrivateSymbol']]
df = df.drop_duplicates()
df = df.reset_index().drop(columns=['index'])
df

,PremiseAddress,PublicSymbol,PrivateSymbol
0,222 W ST NW,NonLead,NonLead
1,436 11TH ST NW,NonLead,NonLead
2,730 12TH ST NW,NonLead,NonLead
3,1425 K ST NW,NonLead,NonLead
4,923 16TH ST NW,NonLead,NonLead
...,...,...,...
110549,3157 12TH ST SE,NonLead,NonLead
110550,1127 SYCAMORE DR SE,NonLead,NonLead
110551,1125 SYCAMORE DR SE,NonLead,NonLead
110552,1123 SYCAMORE DR SE,NonLead,NonLead


In [3]:
#MEDIUM Journal method

import requests
from bs4 import BeautifulSoup

l=list()
obj={}
 
headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
          }
 
resp =  requests.get("https://www.realtor.com/realestateandhomes-search/Washington_DC", headers=headers).text
soup = BeautifulSoup(resp,'html.parser')

In [5]:
import requests
from bs4 import BeautifulSoup

l = list()

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
}

for page_number in range(1, 74):  # Iterate through pages 1 to 74
    url = f"https://www.realtor.com/realestateandhomes-search/Washington_DC/pg-{page_number}"
    
    resp = requests.get(url, headers=headers).text
    soup = BeautifulSoup(resp, 'html.parser')
    
    for el in soup.select(".BasePropertyCard_propertyCardWrap__J0xUj"):
        obj = {}
        try:
            obj["pricing"] = el.select_one(".price-wrapper div[data-testid=card-price]").text
        except:
            obj["pricing"] = None
        try:
            obj["bed"] = el.select_one("li[data-testid=property-meta-beds]").text
        except:
            obj["bed"] = None
        try:
            obj["bath"] = el.select_one("li[data-testid=property-meta-baths]").text
        except:
            obj["bath"] = None
        try:
            obj["sqft"] = el.select_one("li[data-testid=property-meta-sqft]").find_next().text
        except:
            obj["sqft"] = None
        try:
            obj["plot_size"] = el.select_one("li[data-testid=property-meta-lot-size]").find_next().text
        except:
            obj["plot_size"] = None
        try:
            obj["address"] = el.select_one("div[data-testid=card-address-1]").text + " " + el.select_one("div[data-testid=card-address-2]").text
        except:
            obj["address"] = None
        l.append(obj)

print(l)


[{'pricing': '$5,900,000', 'bed': '5bed', 'bath': '5.5bath', 'sqft': '5,478sqft', 'plot_size': '9,102sqft lot', 'address': '3246 S St NW Washington, DC 20007'}, {'pricing': '$1,100,000', 'bed': '5bed', 'bath': '3.5bath', 'sqft': '3,652sqft', 'plot_size': '5,463sqft lot', 'address': '5430 N Capitol St NW Washington, DC 20011'}, {'pricing': '$6,825,000', 'bed': '6bed', 'bath': '6.5bath', 'sqft': '8,011sqft', 'plot_size': '0.41acre lot', 'address': '3065 University Ter NW Washington, DC 20016'}, {'pricing': '$1,695,000', 'bed': '4bed', 'bath': '3.5bath', 'sqft': '1,941sqft', 'plot_size': '1,105sqft lot', 'address': '104 4th St SE Washington, DC 20003'}, {'pricing': '$499,999', 'bed': '4bed', 'bath': '3bath', 'sqft': '1,418sqft', 'plot_size': '3,575sqft lot', 'address': '3214 W St SE Washington, DC 20020'}, {'pricing': '$749,900', 'bed': '5bed', 'bath': '3bath', 'sqft': '1,152sqft', 'plot_size': '1,862sqft lot', 'address': '5604 Kansas Ave NW Washington, DC 20011'}, {'pricing': '$745,000',

In [7]:
housing_df = pd.DataFrame(l)

# Preprocess address columns to make them consistent for merging
housing_df['address'] = housing_df['address'].str.lower().str.strip()
df['PremiseAddress'] = df['PremiseAddress'].str.lower().str.strip()
housing_df['address'] = housing_df['address'].str.replace(r',? washington,? dc \d{5}$', '', regex=True)
housing_df['address'] = housing_df['address'].astype(str)
df['PremiseAddress'] = df['PremiseAddress'].astype(str)
housing_df['address'] = housing_df['address'].str.strip()
df['PremiseAddress'] = df['PremiseAddress'].str.strip()

housing_df

,pricing,bed,bath,sqft,plot_size,address
0,"$5,900,000",5bed,5.5bath,"5,478sqft","9,102sqft lot",3246 s st nw
1,"$1,100,000",5bed,3.5bath,"3,652sqft","5,463sqft lot",5430 n capitol st nw
2,"$6,825,000",6bed,6.5bath,"8,011sqft",0.41acre lot,3065 university ter nw
3,"$1,695,000",4bed,3.5bath,"1,941sqft","1,105sqft lot",104 4th st se
4,"$499,999",4bed,3bath,"1,418sqft","3,575sqft lot",3214 w st se
...,...,...,...,...,...,...
3023,"$1,200,000",4bed,3bath,"2,064sqft","2,888sqft lot",1127 orren st ne
3024,"$1,100,000",3bed,1bath,"1,350sqft","1,151sqft lot",1421 carrollsburg pl sw
3025,"$344,900",1bed,1bath,612sqft,None,1910 benning rd ne unit a
3026,"$270,000",2bed,1bath,"1,030sqft","2,338sqft lot",196 54th st se


In [8]:
# Merge the datasets based on the address columns
merged_df = pd.merge(housing_df, df, left_on='address', right_on='PremiseAddress', how='inner')
merged_df

,pricing,bed,bath,sqft,plot_size,address,PremiseAddress,PublicSymbol,PrivateSymbol
0,"$6,825,000",6bed,6.5bath,"8,011sqft",0.41acre lot,3065 university ter nw,3065 university ter nw,NonLead,NonLead
1,"$1,695,000",4bed,3.5bath,"1,941sqft","1,105sqft lot",104 4th st se,104 4th st se,NonLead,NonLead
2,"$499,999",4bed,3bath,"1,418sqft","3,575sqft lot",3214 w st se,3214 w st se,NonLead,NonLead
3,"$749,900",5bed,3bath,"1,152sqft","1,862sqft lot",5604 kansas ave nw,5604 kansas ave nw,NonLead,NonLead
4,"$749,999",3bed,2.5bath,"1,450sqft","1,180sqft lot",1426 pennsylvania ave se,1426 pennsylvania ave se,Lead,Lead
...,...,...,...,...,...,...,...,...,...
1146,"$2,450,000",4bed,None,"2,322sqft","2,400sqft lot",3113 georgia ave nw,3113 georgia ave nw,Lead,Lead
1147,"$2,000,000",4bed,None,"2,800sqft","4,704sqft lot",2606 41st st nw,2606 41st st nw,NonLead,NonLead
1148,"$900,000",Studio,None,"1,394sqft","1,502sqft lot",113 18th st se,113 18th st se,Lead,Lead
1149,"$2,000,000",4bed,None,"2,800sqft","4,704sqft lot",2610 41st st nw,2610 41st st nw,NonLead,NonLead


In [9]:
merged_df['PublicSymbol'].value_counts()

PublicSymbol
NonLead    1029
Lead        122
Name: count, dtype: int64

In [10]:
merged_df['PrivateSymbol'].value_counts()

PrivateSymbol
NonLead    906
Lead       245
Name: count, dtype: int64

In [11]:
merged_df = merged_df.drop(columns=['PremiseAddress'])
merged_df

,pricing,bed,bath,sqft,plot_size,address,PublicSymbol,PrivateSymbol
0,"$6,825,000",6bed,6.5bath,"8,011sqft",0.41acre lot,3065 university ter nw,NonLead,NonLead
1,"$1,695,000",4bed,3.5bath,"1,941sqft","1,105sqft lot",104 4th st se,NonLead,NonLead
2,"$499,999",4bed,3bath,"1,418sqft","3,575sqft lot",3214 w st se,NonLead,NonLead
3,"$749,900",5bed,3bath,"1,152sqft","1,862sqft lot",5604 kansas ave nw,NonLead,NonLead
4,"$749,999",3bed,2.5bath,"1,450sqft","1,180sqft lot",1426 pennsylvania ave se,Lead,Lead
...,...,...,...,...,...,...,...,...
1146,"$2,450,000",4bed,None,"2,322sqft","2,400sqft lot",3113 georgia ave nw,Lead,Lead
1147,"$2,000,000",4bed,None,"2,800sqft","4,704sqft lot",2606 41st st nw,NonLead,NonLead
1148,"$900,000",Studio,None,"1,394sqft","1,502sqft lot",113 18th st se,Lead,Lead
1149,"$2,000,000",4bed,None,"2,800sqft","4,704sqft lot",2610 41st st nw,NonLead,NonLead


In [25]:
#More Precossessing
merged_df['bed'] = merged_df['bed'].str.replace('bed','')
merged_df['bath'] = merged_df['bath'].str.replace('bath','')
merged_df['sqft'] = merged_df['sqft'].str.replace('sqft','')
merged_df['plot_size'] = merged_df['plot_size'].str.replace('sqft lot','')
merged_df['plot_size'] = merged_df['plot_size'].str.replace('acre lot','')
merged_df

,pricing,bed,bath,sqft,plot_size,address,PublicSymbol,PrivateSymbol
0,"$6,825,000",6,6.5,"8,011",0.41,3065 university ter nw,NonLead,NonLead
1,"$1,695,000",4,3.5,"1,941","1,105",104 4th st se,NonLead,NonLead
2,"$499,999",4,3,"1,418","3,575",3214 w st se,NonLead,NonLead
3,"$749,900",5,3,"1,152","1,862",5604 kansas ave nw,NonLead,NonLead
4,"$749,999",3,2.5,"1,450","1,180",1426 pennsylvania ave se,Lead,Lead
...,...,...,...,...,...,...,...,...
1146,"$2,450,000",4,None,"2,322","2,400",3113 georgia ave nw,Lead,Lead
1147,"$2,000,000",4,None,"2,800","4,704",2606 41st st nw,NonLead,NonLead
1148,"$900,000",Studio,None,"1,394","1,502",113 18th st se,Lead,Lead
1149,"$2,000,000",4,None,"2,800","4,704",2610 41st st nw,NonLead,NonLead


In [27]:
merged_df.to_csv('PreliminaryDCdataset.csv', index=False) 